# Power Loss Calculation

This notebook covers power loss analysis in Pulsim for power electronics efficiency estimation.

## Contents
1. Power Loss Fundamentals
2. Conduction Losses
3. Switching Losses
4. Diode Reverse Recovery
5. Loss Breakdown by Device
6. Efficiency Calculation
7. Thermal Coupling

In [ ]:
import pulsim
import numpy as np
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = [12, 5]
plt.rcParams['font.size'] = 11

## 1. Power Loss Fundamentals

Total power losses in switching converters:

$$P_{total} = P_{conduction} + P_{switching} + P_{gate} + P_{other}$$

### Conduction Losses
$$P_{cond} = I_{rms}^2 \cdot R_{ds(on)} \quad \text{(MOSFET)}$$
$$P_{cond} = V_{CE(sat)} \cdot I_{avg} + I_{rms}^2 \cdot R_{CE} \quad \text{(IGBT)}$$
$$P_{cond} = V_f \cdot I_{avg} \quad \text{(Diode)}$$

### Switching Losses
$$P_{sw} = \frac{1}{2} V_{DS} \cdot I_D \cdot (t_{rise} + t_{fall}) \cdot f_{sw}$$

### Reverse Recovery Losses
$$P_{rr} = \frac{1}{2} Q_{rr} \cdot V_R \cdot f_{sw}$$

In [ ]:
# Power loss data structure
losses = pulsim.PowerLosses()

print("PowerLosses structure:")
print(f"  conduction_loss: Total conduction energy (J)")
print(f"  turn_on_loss: Total turn-on switching energy (J)")
print(f"  turn_off_loss: Total turn-off switching energy (J)")
print(f"  reverse_recovery_loss: Total Qrr energy (J)")
print(f"\nMethods:")
print(f"  switching_loss(): turn_on + turn_off")
print(f"  total_loss(): conduction + switching + reverse_recovery")

## 2. Conduction Losses

Conduction losses are calculated from the instantaneous power:

$$P_{cond}(t) = V(t) \cdot I(t)$$

Integrated over the simulation period.

In [ ]:
# MOSFET conduction loss example
Rds_on = 0.01      # 10mΩ
I_load = 10        # 10A
D = 0.5            # 50% duty cycle
f_sw = 100e3       # 100kHz

# RMS current through high-side MOSFET
I_rms_hs = I_load * np.sqrt(D)

# Conduction loss
P_cond_hs = I_rms_hs**2 * Rds_on

print("MOSFET Conduction Loss (High-Side):")
print(f"  Load current: {I_load} A")
print(f"  Duty cycle: {D*100:.0f}%")
print(f"  RMS current: {I_rms_hs:.2f} A")
print(f"  Rds_on: {Rds_on*1000:.0f} mΩ")
print(f"  Conduction loss: {P_cond_hs:.2f} W")

In [ ]:
# Simulate buck converter and extract losses
netlist = f'''
{{
  "name": "Buck Converter Loss Analysis",
  "components": [
    {{"type": "V", "name": "Vin", "nodes": ["vin", "0"], "value": 12}},
    {{"type": "V", "name": "Vpwm_h", "nodes": ["pwm_h", "0"],
     "waveform": {{"type": "pwm", "v_off": 0, "v_on": 10, 
                  "frequency": {f_sw}, "duty": {D}, "dead_time": 100e-9}}}},
    {{"type": "V", "name": "Vpwm_l", "nodes": ["pwm_l", "0"],
     "waveform": {{"type": "pwm", "v_off": 0, "v_on": 10, 
                  "frequency": {f_sw}, "duty": {D}, "dead_time": 100e-9,
                  "complementary": true}}}},
    
    {{"type": "M", "name": "M_hs", "nodes": ["vin", "pwm_h", "sw"],
     "params": {{"type": "nmos", "vth": 2.0, "rds_on": {Rds_on}}}}},
    {{"type": "M", "name": "M_ls", "nodes": ["sw", "pwm_l", "0"],
     "params": {{"type": "nmos", "vth": 2.0, "rds_on": {Rds_on}}}}},
    
    {{"type": "L", "name": "L1", "nodes": ["sw", "out"], "value": 100e-6}},
    {{"type": "C", "name": "C1", "nodes": ["out", "0"], "value": 100e-6}},
    {{"type": "R", "name": "Rload", "nodes": ["out", "0"], "value": 0.6}}
  ]
}}
'''

circuit = pulsim.parse_netlist_string(netlist)

options = pulsim.SimulationOptions()
options.tstop = 500e-6  # 50 switching cycles
options.dt = 10e-9

result = pulsim.simulate(circuit, options)
data = result.to_dict()

# Get power losses from simulator
losses = result.power_losses() if hasattr(result, 'power_losses') else None

time_us = np.array(data['time']) * 1e6
v_out = np.array(data['signals']['V(out)'])
v_sw = np.array(data['signals']['V(sw)'])

# Calculate output power
v_out_avg = np.mean(v_out[-2000:])
i_out = v_out_avg / 0.6
p_out = v_out_avg * i_out

print(f"\nSimulation Results:")
print(f"  Output voltage: {v_out_avg:.2f} V")
print(f"  Output current: {i_out:.2f} A")
print(f"  Output power: {p_out:.2f} W")

In [ ]:
# Manual conduction loss calculation from waveforms
# Use steady-state portion
ss_start = len(time_us) // 2

# Calculate instantaneous power dissipation in switches
# P = V_ds * I_d
v_ds_hs = np.array(data['signals']['V(vin)']) - np.array(data['signals']['V(sw)'])
v_ds_ls = np.array(data['signals']['V(sw)'])

# Inductor current (approximate)
i_L = (v_out_avg / 0.6) * np.ones_like(v_ds_hs)  # Simplified

# Power in high-side (when on)
p_hs = v_ds_hs * i_L
p_hs_avg = np.mean(p_hs[ss_start:])

# Power in low-side (when on)
p_ls = np.abs(v_ds_ls) * i_L  
p_ls_avg = np.mean(p_ls[ss_start:])

print(f"Estimated Power Dissipation:")
print(f"  High-side MOSFET: {p_hs_avg:.3f} W")
print(f"  Low-side MOSFET: {p_ls_avg:.3f} W")

## 3. Switching Losses

Switching losses occur during the voltage-current overlap:

$$E_{sw} = \int_0^{t_{sw}} v(t) \cdot i(t) \, dt$$

For linear transitions:
$$E_{on} \approx \frac{1}{2} V_{DS} \cdot I_D \cdot t_{on}$$
$$E_{off} \approx \frac{1}{2} V_{DS} \cdot I_D \cdot t_{off}$$

In [ ]:
# Switching loss calculation
V_ds = 12          # Drain-source voltage
I_d = 10           # Drain current
t_on = 20e-9       # Turn-on time (20ns)
t_off = 30e-9      # Turn-off time (30ns)
f_sw = 100e3       # Switching frequency

# Energy per switching event
E_on = 0.5 * V_ds * I_d * t_on
E_off = 0.5 * V_ds * I_d * t_off
E_sw_total = E_on + E_off

# Power (energy × frequency)
P_sw = E_sw_total * f_sw

print("Switching Loss Calculation:")
print(f"  V_ds: {V_ds} V")
print(f"  I_d: {I_d} A")
print(f"  Turn-on time: {t_on*1e9:.0f} ns")
print(f"  Turn-off time: {t_off*1e9:.0f} ns")
print(f"\n  Turn-on energy: {E_on*1e6:.2f} µJ")
print(f"  Turn-off energy: {E_off*1e6:.2f} µJ")
print(f"  Total switching energy: {E_sw_total*1e6:.2f} µJ")
print(f"\n  Switching power @ {f_sw/1e3:.0f}kHz: {P_sw:.2f} W")

In [ ]:
# Visualize switching waveforms
# Idealized turn-on transition
t_sw = np.linspace(0, 100e-9, 1000)

# Turn-on: current rises first, then voltage falls
t_rise_i = 10e-9
t_fall_v = 20e-9

i_on = np.where(t_sw < t_rise_i, 
                I_d * t_sw / t_rise_i,
                I_d)

v_on = np.where(t_sw < t_rise_i,
                V_ds,
                np.where(t_sw < t_rise_i + t_fall_v,
                        V_ds * (1 - (t_sw - t_rise_i) / t_fall_v),
                        0))

p_on = v_on * i_on

fig, axes = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

# V and I
axes[0].plot(t_sw * 1e9, v_on, 'b-', linewidth=2, label='V_ds')
axes[0].plot(t_sw * 1e9, i_on, 'r-', linewidth=2, label='I_d')
axes[0].set_ylabel('V (V) / I (A)')
axes[0].set_title('MOSFET Turn-On Transition')
axes[0].legend()
axes[0].grid(True)
axes[0].fill_between(t_sw * 1e9, 0, v_on, alpha=0.2, color='blue')
axes[0].fill_between(t_sw * 1e9, 0, i_on, alpha=0.2, color='red')

# Power
axes[1].plot(t_sw * 1e9, p_on, 'g-', linewidth=2)
axes[1].fill_between(t_sw * 1e9, 0, p_on, alpha=0.3, color='green')
axes[1].set_xlabel('Time (ns)')
axes[1].set_ylabel('Instantaneous Power (W)')
axes[1].set_title(f'Switching Power Loss (shaded area = {np.trapz(p_on, t_sw)*1e6:.2f} µJ)')
axes[1].grid(True)

plt.tight_layout()
plt.show()

## 4. Diode Reverse Recovery

When a diode turns off, it conducts reverse current briefly:

$$Q_{rr} = \int I_{rr}(t) \, dt$$

$$E_{rr} = \frac{1}{2} Q_{rr} \cdot V_R$$

$$P_{rr} = E_{rr} \cdot f_{sw}$$

Reverse recovery causes:
- Additional losses in diode and switch
- Current/voltage spikes
- EMI

In [ ]:
# Reverse recovery loss calculation
# Typical parameters for fast recovery diode
Q_rr = 50e-9       # 50nC reverse recovery charge
V_R = 12           # Reverse voltage
t_rr = 30e-9       # Reverse recovery time
f_sw = 100e3

# Energy per event
E_rr = 0.5 * Q_rr * V_R

# Power
P_rr = E_rr * f_sw

# Peak reverse current
I_rr_peak = 2 * Q_rr / t_rr

print("Diode Reverse Recovery:")
print(f"  Qrr: {Q_rr*1e9:.0f} nC")
print(f"  trr: {t_rr*1e9:.0f} ns")
print(f"  Peak Irr: {I_rr_peak:.2f} A")
print(f"\n  Energy per event: {E_rr*1e9:.1f} nJ")
print(f"  Power @ {f_sw/1e3:.0f}kHz: {P_rr*1e3:.1f} mW")

In [ ]:
# Visualize reverse recovery waveform
t = np.linspace(0, 100e-9, 1000)

# Simplified reverse recovery current waveform
t_a = t_rr * 0.3   # Time to peak Irr
t_b = t_rr * 0.7   # Recovery time

I_f = 5  # Forward current before turn-off
di_dt = 200e6  # Current slope (A/s)

# Piecewise waveform
i_rr = np.piecewise(t,
    [t < t_a, (t >= t_a) & (t < t_a + t_b), t >= t_a + t_b],
    [lambda t: I_f - di_dt * t,
     lambda t: -I_rr_peak * (1 - (t - t_a) / t_b),
     0])

plt.figure(figsize=(12, 5))
plt.plot(t * 1e9, i_rr, 'b-', linewidth=2)
plt.axhline(y=0, color='k', linestyle='-', linewidth=0.5)
plt.fill_between(t * 1e9, 0, np.minimum(i_rr, 0), alpha=0.3, color='red', 
                 label=f'Qrr area')
plt.xlabel('Time (ns)')
plt.ylabel('Diode Current (A)')
plt.title('Diode Reverse Recovery Waveform')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

print("Note: Qrr is the integral of negative current (shaded area)")

## 5. Loss Breakdown by Device

Pulsim tracks losses per device for detailed analysis.

In [ ]:
# Complete loss analysis for a buck converter
# Design parameters
Vin = 12
Vout = 5
Iout = 10
f_sw = 100e3
D = Vout / Vin

# MOSFET parameters
Rds_on = 0.010     # 10mΩ
t_on = 20e-9
t_off = 30e-9

# Diode parameters (for async version)
Vf = 0.5           # Forward voltage
Q_rr = 20e-9       # 20nC

# Calculate losses
print("Buck Converter Loss Analysis")
print("=" * 50)
print(f"Vin = {Vin}V, Vout = {Vout}V, Iout = {Iout}A")
print(f"Duty cycle D = {D:.2f}, fsw = {f_sw/1e3:.0f}kHz")
print()

# High-side MOSFET
I_rms_hs = Iout * np.sqrt(D)
P_cond_hs = I_rms_hs**2 * Rds_on
P_sw_hs = 0.5 * Vin * Iout * (t_on + t_off) * f_sw
P_total_hs = P_cond_hs + P_sw_hs

print("High-Side MOSFET:")
print(f"  Irms = {I_rms_hs:.2f} A")
print(f"  Conduction loss = {P_cond_hs:.3f} W")
print(f"  Switching loss = {P_sw_hs:.3f} W")
print(f"  Total = {P_total_hs:.3f} W")
print()

# Low-side: Compare diode vs sync MOSFET
print("Low-Side Options:")

# Option 1: Diode
P_cond_diode = Vf * Iout * (1 - D)
P_rr = 0.5 * Q_rr * Vin * f_sw
P_total_diode = P_cond_diode + P_rr

print(f"  Diode:")
print(f"    Conduction = {P_cond_diode:.3f} W")
print(f"    Reverse recovery = {P_rr:.3f} W")
print(f"    Total = {P_total_diode:.3f} W")

# Option 2: Synchronous MOSFET
I_rms_ls = Iout * np.sqrt(1 - D)
P_cond_ls = I_rms_ls**2 * Rds_on
P_sw_ls = 0  # Assume ZVS due to body diode
P_total_sync = P_cond_ls + P_sw_ls

print(f"  Sync MOSFET:")
print(f"    Conduction = {P_cond_ls:.3f} W")
print(f"    Switching = {P_sw_ls:.3f} W (ZVS)")
print(f"    Total = {P_total_sync:.3f} W")
print()

# Total system
P_out = Vout * Iout
P_loss_diode = P_total_hs + P_total_diode
P_loss_sync = P_total_hs + P_total_sync

eff_diode = P_out / (P_out + P_loss_diode) * 100
eff_sync = P_out / (P_out + P_loss_sync) * 100

print("System Efficiency:")
print(f"  Output power = {P_out:.1f} W")
print(f"  With diode: {eff_diode:.1f}% (losses = {P_loss_diode:.2f} W)")
print(f"  Synchronous: {eff_sync:.1f}% (losses = {P_loss_sync:.2f} W)")

In [ ]:
# Visualize loss breakdown
components = ['HS Conduction', 'HS Switching', 'LS Conduction', 'LS Switching/Qrr']

losses_diode = [P_cond_hs, P_sw_hs, P_cond_diode, P_rr]
losses_sync = [P_cond_hs, P_sw_hs, P_cond_ls, 0]

x = np.arange(len(components))
width = 0.35

fig, ax = plt.subplots(figsize=(12, 6))
bars1 = ax.bar(x - width/2, losses_diode, width, label='Diode Rectification', color='coral')
bars2 = ax.bar(x + width/2, losses_sync, width, label='Synchronous Rectification', color='steelblue')

ax.set_ylabel('Power Loss (W)')
ax.set_title('Loss Breakdown: Diode vs Synchronous Rectification')
ax.set_xticks(x)
ax.set_xticklabels(components)
ax.legend()
ax.grid(True, axis='y', alpha=0.5)

# Add value labels
for bar in bars1:
    height = bar.get_height()
    if height > 0.01:
        ax.annotate(f'{height:.2f}W',
                    xy=(bar.get_x() + bar.get_width()/2, height),
                    ha='center', va='bottom', fontsize=9)

for bar in bars2:
    height = bar.get_height()
    if height > 0.01:
        ax.annotate(f'{height:.2f}W',
                    xy=(bar.get_x() + bar.get_width()/2, height),
                    ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

## 6. Efficiency Calculation

$$\eta = \frac{P_{out}}{P_{in}} = \frac{P_{out}}{P_{out} + P_{losses}}$$

In [ ]:
# Efficiency vs load current
I_loads = np.linspace(0.5, 15, 30)

eff_diode_arr = []
eff_sync_arr = []

for I in I_loads:
    # High-side losses
    P_cond_hs = (I * np.sqrt(D))**2 * Rds_on
    P_sw_hs = 0.5 * Vin * I * (t_on + t_off) * f_sw
    
    # Low-side diode
    P_cond_d = Vf * I * (1 - D)
    P_rr_d = 0.5 * Q_rr * Vin * f_sw
    
    # Low-side sync
    P_cond_s = (I * np.sqrt(1 - D))**2 * Rds_on
    
    # Output
    P_out = Vout * I
    
    # Efficiencies
    P_loss_d = P_cond_hs + P_sw_hs + P_cond_d + P_rr_d
    P_loss_s = P_cond_hs + P_sw_hs + P_cond_s
    
    eff_diode_arr.append(P_out / (P_out + P_loss_d) * 100)
    eff_sync_arr.append(P_out / (P_out + P_loss_s) * 100)

plt.figure(figsize=(12, 6))
plt.plot(I_loads, eff_diode_arr, 'r-', linewidth=2, label='Diode Rectification')
plt.plot(I_loads, eff_sync_arr, 'b-', linewidth=2, label='Synchronous Rectification')
plt.axhline(y=90, color='gray', linestyle='--', alpha=0.5)
plt.xlabel('Load Current (A)')
plt.ylabel('Efficiency (%)')
plt.title('Buck Converter Efficiency vs Load Current')
plt.legend()
plt.grid(True)
plt.ylim([80, 100])
plt.xlim([0, 15])
plt.tight_layout()
plt.show()

print("Synchronous rectification provides higher efficiency,")
print("especially at high currents where I²R losses dominate.")

In [ ]:
# Efficiency vs switching frequency
freqs = np.logspace(4, 6, 30)  # 10kHz to 1MHz
I = 10  # Fixed 10A load

eff_vs_freq = []
P_sw_vs_freq = []
P_cond_vs_freq = []

for f in freqs:
    P_cond = (I * np.sqrt(D))**2 * Rds_on + (I * np.sqrt(1-D))**2 * Rds_on
    P_sw = 0.5 * Vin * I * (t_on + t_off) * f
    P_out = Vout * I
    
    eff_vs_freq.append(P_out / (P_out + P_cond + P_sw) * 100)
    P_sw_vs_freq.append(P_sw)
    P_cond_vs_freq.append(P_cond)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Efficiency
axes[0].semilogx(freqs/1e3, eff_vs_freq, 'b-', linewidth=2)
axes[0].set_xlabel('Switching Frequency (kHz)')
axes[0].set_ylabel('Efficiency (%)')
axes[0].set_title('Efficiency vs Switching Frequency')
axes[0].grid(True, which='both', alpha=0.5)
axes[0].set_ylim([85, 100])

# Loss breakdown
axes[1].semilogx(freqs/1e3, P_cond_vs_freq, 'g-', linewidth=2, label='Conduction')
axes[1].semilogx(freqs/1e3, P_sw_vs_freq, 'r-', linewidth=2, label='Switching')
axes[1].semilogx(freqs/1e3, np.array(P_cond_vs_freq) + np.array(P_sw_vs_freq), 
                  'k--', linewidth=2, label='Total')
axes[1].set_xlabel('Switching Frequency (kHz)')
axes[1].set_ylabel('Power Loss (W)')
axes[1].set_title('Power Losses vs Switching Frequency')
axes[1].legend()
axes[1].grid(True, which='both', alpha=0.5)

plt.tight_layout()
plt.show()

print("Higher frequency → More switching losses → Lower efficiency")
print("But allows smaller passives (L, C)")

## 7. Thermal Coupling

Power losses cause device heating. With thermal-electrical coupling:
- Rds_on increases with temperature
- Higher Rds_on → More losses → Higher temperature
- Positive feedback loop!

In [ ]:
# Thermal-electrical coupling analysis
# Rds_on temperature coefficient
Rds_on_25C = 0.010  # 10mΩ at 25°C
TC = 0.007          # 0.7%/°C positive temp coefficient

# Thermal resistance
Rth_jc = 1.0        # Junction to case (°C/W)
Rth_cs = 0.5        # Case to sink
Rth_sa = 2.0        # Sink to ambient
Rth_total = Rth_jc + Rth_cs + Rth_sa

T_ambient = 25      # °C

# Iterative solution for steady-state temperature
def solve_thermal_equilibrium(I_rms, D, T_amb):
    T_j = T_amb  # Initial guess
    
    for _ in range(20):  # Iterate
        # Temperature-dependent Rds_on
        Rds = Rds_on_25C * (1 + TC * (T_j - 25))
        
        # Power dissipation
        P_loss = I_rms**2 * Rds
        
        # Junction temperature
        T_j_new = T_amb + P_loss * Rth_total
        
        if abs(T_j_new - T_j) < 0.1:
            break
        T_j = T_j_new
    
    return T_j, Rds, P_loss

# Analyze for different currents
I_range = np.linspace(1, 20, 20)
T_j_arr = []
Rds_arr = []
P_loss_arr = []

for I in I_range:
    I_rms = I * np.sqrt(0.5)  # 50% duty cycle
    T_j, Rds, P = solve_thermal_equilibrium(I_rms, 0.5, T_ambient)
    T_j_arr.append(T_j)
    Rds_arr.append(Rds * 1000)  # Convert to mΩ
    P_loss_arr.append(P)

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

axes[0].plot(I_range, T_j_arr, 'r-', linewidth=2)
axes[0].axhline(y=150, color='orange', linestyle='--', label='Max Tj (150°C)')
axes[0].set_xlabel('Load Current (A)')
axes[0].set_ylabel('Junction Temperature (°C)')
axes[0].set_title('Junction Temperature vs Current')
axes[0].legend()
axes[0].grid(True)

axes[1].plot(I_range, Rds_arr, 'b-', linewidth=2)
axes[1].axhline(y=Rds_on_25C*1000, color='g', linestyle='--', label='Rds_on @ 25°C')
axes[1].set_xlabel('Load Current (A)')
axes[1].set_ylabel('Rds_on (mΩ)')
axes[1].set_title('Rds_on Increase with Temperature')
axes[1].legend()
axes[1].grid(True)

axes[2].plot(I_range, P_loss_arr, 'g-', linewidth=2)
# Also plot losses without thermal coupling
P_no_thermal = [(I * np.sqrt(0.5))**2 * Rds_on_25C for I in I_range]
axes[2].plot(I_range, P_no_thermal, 'g--', linewidth=1.5, alpha=0.7, label='No thermal coupling')
axes[2].set_xlabel('Load Current (A)')
axes[2].set_ylabel('Power Loss (W)')
axes[2].set_title('Power Loss (with thermal feedback)')
axes[2].legend()
axes[2].grid(True)

plt.tight_layout()
plt.show()

print("Thermal coupling increases losses above the cold calculation!")

## Summary

### Loss Components

| Loss Type | Formula | Dominant Factor |
|-----------|---------|----------------|
| Conduction | I²R | Current (high at high loads) |
| Switching | V×I×t×f | Frequency |
| Reverse Recovery | Qrr×V×f | Diode type, frequency |
| Gate Drive | Qg×Vg×f | Gate charge, frequency |

### Optimization Guidelines

1. **Low current, high frequency**: Switching losses dominate
   - Use low-capacitance devices
   - Consider ZVS/ZCS techniques

2. **High current, low frequency**: Conduction losses dominate
   - Use low Rds_on MOSFETs (larger die)
   - Parallel devices

3. **General**:
   - Use synchronous rectification for high efficiency
   - Account for thermal effects on Rds_on
   - Balance switching vs conduction losses

### Pulsim Loss Analysis

```python
result = pulsim.simulate(circuit, options)
losses = result.power_losses()

print(f"Conduction: {losses.conduction_loss} J")
print(f"Switching: {losses.switching_loss()} J")
print(f"Total: {losses.total_loss()} J")
```

**Next:** [SPICE Parser and Netlists](12_spice_parser.ipynb)